In [10]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import os
from datetime import datetime
from scipy.stats import norm

import utils as ut
from improc import *
import worm_env as we
import policy_time as pt

# R file: get_posterior_bart.R should be running simultaneously.

### First worm

In [2]:
def gen_pol_collection(
    worm_id, 
    
    # All parameters below this should be same as in first_pol_collection() function.
    folder, # Remember to make it first
    worm_ep_len=60,
    tot_eps=5,
    base_sampling=None, # A vector of len(tot_eps) that says the fraction of time to sample a 
                        # state that isn't marked high-entropy. 1 means always sample (light on 50% of time)
    samp_limit=1, # Proportion of time we want to sample. Goes from 0 to 1. A 1 means light is on 50% of time.
    ):
    
    # Fixed parameters
    params = {
        'reward_ahead': 30,
        'timestep_gap': 1,
        'prev_act_window': 3,
        'jump_limit': 100,
    }
    if base_sampling is None:
        base_sampling = np.ones(tot_eps)
    
    # Load entropies and dataframe from previous run    
    curr_ep = worm_id*tot_eps
    dh = pt.DataHandler(params=params)
    
    if curr_ep==0:
        # init pol 
        counts, ents = np.ones((12,12)), np.ones((12,12))
    else:    
        dh.load_df(f'{folder}df{curr_ep-1}.pkl')
        ents = np.load(f'{folder}ents{curr_ep-1}.npy',allow_pickle=True)
        counts = pt.get_counts(dh.df)
    
    worm = we.ProcessedWorm(0,ep_len=worm_ep_len)

    for i,ep in enumerate(np.arange(tot_eps)+curr_ep):
        # Create a list of states to search based on entropy and cutoff.
        # Note that samp_limit*.5 is proportion of time light is on.
        sampling_probs = pt.make_sprobs_cutoff(ents,samp_limit,counts,base_sampling[i])


        # Get new data
        fnames = []
        fnames.append(f'{folder}traj{ep}.pkl')
        pt.do_sampling_traj(sampling_probs, fnames[0], worm, 1, act_rate=3)


        # Save for R in addition to old data 
        dh_n = pt.DataHandler(params=params)
        dh_n.add_dict_to_df(fnames)
        dh_n.df = dh_n.df.append(dh.df, ignore_index=True)
        dh_n.save_dfs(f'{folder}df{ep}.pkl')
        pt.save_for_R(dh_n.df, f'{folder}traj{ep}.npy')    
        dh = dh_n


        # Wait for R script to run
        bart_f = f'{folder}sbart{ep}.npy'
        while not os.path.exists(bart_f):
            time.sleep(1)
        # Get the probabilities and entropies 
        post = np.load(bart_f,allow_pickle=True)
        probs, ents = pt.bart2pols(post)
        np.save(f'{folder}probs{ep}.npy',probs,allow_pickle=True)
        np.save(f'{folder}ents{ep}.npy',ents,allow_pickle=True)

In [5]:
folder = './Data/03_08_0/'
gen_pol_collection(5,folder,worm_ep_len=300,tot_eps=5)

Done resetting	
Saved as numpy for R
Done resetting	
Saved as numpy for R
Done resetting	
Saved as numpy for R
Done resetting	
Saved as numpy for R
Done resetting	
Saved as numpy for R


In [18]:
def f(**kwargs):
    for key,val in kwargs.items():
        print(key,val)

In [20]:
f(a=1,b=2)

a 1
b 2


In [21]:
args = {'a':1,'b':2}

In [25]:
f(**args)

a 1
b 2


In [27]:
if 3 in range(3): print('s')